## 构建一个RAG应用
> RAG：Retrieval Augmented Generation

**官方文档：** https://python.langchain.com/docs/tutorials/chatbot/

构建一个RAG简单应用主要包含两部分：文档索引和查询生成

### 第一部分：文档索引

#### 参考文档
1. https://python.langchain.com/docs/tutorials/retrievers/

#### 核心步骤
1. 加载原始数据（非结构化的文本数据）
2. 切分文本数据
3. 存储处理后的数据

#### 第一步：加载原始数据
数据来源CSV文件，其它类型的数据加载可以参考文档：https://python.langchain.com/docs/how_to/#document-loaders

In [23]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 相对路径
csv_file_path = "./knowledge-documents.csv"
loader = CSVLoader(
    file_path=csv_file_path,
    content_columns=["知识项"],
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        # "fieldnames": ["知识项"]
    }
)
content = loader.load()

In [24]:
content

[Document(metadata={'source': './knowledge-documents.csv', 'row': 0}, page_content='知识项: 为了验证CodeAct的有效性，本文进行了一系列实验，比较了CodeAct、JSON和文本格式在工具调用任务中的表现。实验结果表明，CodeAct在大多数LLM上表现优于或至少不逊色于JSON和文本格式。特别是在开源模型上，CodeAct的优势更为明显，因为开源模型在预训练阶段接触了大量的代码数据，使得它们更容易适应CodeAct的格式。'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 1}, page_content='知识项: MCP（Model Context Protocol，模型上下文协议）是由 Anthropic 在 2024 年底推出的一种开放协议，旨在通过标准化接口实现大语言模型（LLM）与外部数据源及工具的无缝集成。我们可以这样理解 MCP，它就像是 AI 应用领域的 USB-C 接口。正如 USB-C 为各种设备提供了统一的连接方式，MCP 也为 AI 模型与不同数据源和工具之间提供了一种标准化的连接方式，这也极大地提高了用户的体验和效率。之所以发布 MCP，Anthropic 在一篇博客中表示随着 AI 助手越来越获得主流机构采用，行业在模型能力上投入巨大，推理和质量方面取得了快速进步。然而，即使是最复杂的模型也受到其与数据隔离的限制 —— 被困在信息孤岛和遗留系统中。每个新的数据源都需要自己的定制实现，使得真正连接的系统难以扩展。MCP 解决了这一挑战。它提供了一个通用的开放标准，用于将 AI 系统与数据源连接起来，用单一协议取代了分散的集成。结果是为 AI 系统提供了一种更简单、更可靠的方式，以获取它们所需的数据。'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 2}, page_content='知识项: 第一罪(FP1)：内容缺失（Missing Content）。提问的问题，无法在被检索文档库中找到，最准确的答案是缺失的。理想情况下，RAG系统回应应该是“抱歉，我不知道答案”。然而，

#### 第二步：切分文本数据
RecursiveCharacterTextSplitter做了什么？为什么要做文本分割？
TextSplitter是做文本数据的分割

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
doc_splits = text_splitter.split_documents(content)

In [20]:
doc_splits

[Document(metadata={'source': './knowledge-documents.csv', 'row': 0}, page_content='知识项:'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 0}, page_content='为了验证CodeAct的有效性，本文进行了一系列实验，比较了CodeAct、JSON和文本格式在工具调用任务中的表现。实验结果表明，CodeAct在大多数LLM上表现优于或至少不逊色于JSON和文本'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 0}, page_content='M上表现优于或至少不逊色于JSON和文本格式。特别是在开源模型上，CodeAct的优势更为明显，因为开源模型在预训练阶段接触了大量的代码数据，使得它们更容易适应CodeAct的格式。'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 1}, page_content='知识项: MCP（Model Context Protocol，模型上下文协议）是由 Anthropic 在 2024'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 1}, page_content='Anthropic 在 2024 年底推出的一种开放协议，旨在通过标准化接口实现大语言模型（LLM）与外部数据源及工具的无缝集成。我们可以这样理解 MCP，它就像是 AI 应用领域的 USB-C'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 1}, page_content='AI 应用领域的 USB-C 接口。正如 USB-C 为各种设备提供了统一的连接方式，MCP 也为 AI'),
 Document(metadata={'source': './knowledge-documents.csv', 'row': 1}, 

#### 第三步：存储处理后的数据

### 第二部分：查询生成